In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import expr
from pyspark.ml.stat import Summarizer
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Flight_Delay_Jan_2020_ontime.csv")

In [ ]:
spark = (SparkSession
       .builder
       .appName("FlightDataStatistics")
       .getOrCreate())

In [ ]:
pysparkDF = spark.read.option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", ",") \
        .csv(inputFile)
        
pysparkDF.printSchema()

### Remove faulty features

In [ ]:
pysparkDF = pysparkDF.drop('_c21')
pysparkDF.printSchema()

### Remove records containing NULL values

In [ ]:
pysparkDF_nonull = pysparkDF.dropna()
count_pysparkDF = pysparkDF.count()
count_pysparkDF_nonull = pysparkDF_nonull.count()

print("Count DF records: " + str(count_pysparkDF))
print("Count DF records with removed NULL values: " + str(count_pysparkDF_nonull))

f"Removed {count_pysparkDF-count_pysparkDF_nonull} records containing NULL values"

### Build String indexer for TAIL_NUM

In [ ]:
tailNum_Indexer = StringIndexer().setInputCol("TAIL_NUM").setOutputCol("TAIL_NUM_ID").fit(pysparkDF_nonull)
pysparkDF_indexed = tailNum_Indexer.transform(pysparkDF_nonull)

### Define label columns

In [ ]:
labelCols = ["DEP_DEL15","ARR_DEL15","CANCELLED","DIVERTED"]
labelCols

### Remove redundant features and labels for unconditional prediction
-> Unconditional is referring to predicting each of the labels without having information on the current status of the flight (Use-Case: Checking the day before)

In [ ]:
# Remove strings from id/string pairs (redundant)
# Remark: since in this dataset both string and id exist already, no further preprocessing via string_indexer is necessary. 
#         Otherwise, strings would have first been converted to ids via string_indexer.
featureCols_unconditional = pysparkDF_indexed.columns.copy()
featureCols_unconditional.remove("TAIL_NUM") # -> TAIL_NUM_ID
featureCols_unconditional.remove("OP_UNIQUE_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_unconditional.remove("OP_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_unconditional.remove("ORIGIN") # -> ORIGIN_AIRPORT_ID
featureCols_unconditional.remove("ORIGIN_AIRPORT_SEQ_ID") # -> ORIGIN_AIRPORT_ID
featureCols_unconditional.remove("DEST") # -> DEST_AIRPORT_ID
featureCols_unconditional.remove("DEST_AIRPORT_SEQ_ID") # -> DEST_AIRPORT_ID
featureCols_unconditional.remove("DEP_TIME_BLK") # -> preliminary elimination, check if model works better with binned values or not

for label in labelCols:
    featureCols_unconditional.remove(label)
                                     
featureCols_unconditional                         

### Remove redundant features and labels for conditional prediction
-> Conditional is referring to predicting each of the labels considering available real-time information on the current status of the flight (Use-Case: Checking while at the airport, pre-flight)

One would expect that prediction performance is increased when the model is aware of the current flight status (=DEP_DEL15)

Example: If the model is aware that the flight has departure delay, it might be able to better predict whether it will also be delayed at arrival

In [ ]:
# Remove strings from id/string pairs (redundant)
# Remark: since in this dataset both string and id exist already, no further preprocessing via string_indexer is necessary. 
#         Otherwise, strings would have first been converted to ids via string_indexer.
featureCols_conditional = pysparkDF_indexed.columns.copy()
featureCols_conditional.remove("TAIL_NUM") # -> TAIL_NUM_ID
featureCols_conditional.remove("OP_UNIQUE_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_conditional.remove("OP_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_conditional.remove("ORIGIN") # -> ORIGIN_AIRPORT_ID
featureCols_conditional.remove("ORIGIN_AIRPORT_SEQ_ID") # -> ORIGIN_AIRPORT_ID
featureCols_conditional.remove("DEST") # -> DEST_AIRPORT_ID
featureCols_conditional.remove("DEST_AIRPORT_SEQ_ID") # -> DEST_AIRPORT_ID
featureCols_conditional.remove("DEP_TIME_BLK") # -> preliminary elimination, check if model works better with binned values or not

for label in [label for label in labelCols if label!="DEP_DEL15"]:
    featureCols_conditional.remove(label)
    
featureCols_conditional

### Build and apply feature column assembler for both featureCols

In [ ]:
assembler_unconditional =  VectorAssembler(outputCol="features", inputCols=list(featureCols_unconditional))
assembler_conditional =  VectorAssembler(outputCol="features", inputCols=list(featureCols_conditional))

featureSet_unconditional = assembler_unconditional.transform(pysparkDF_indexed)
featureSet_conditional = assembler_conditional.transform(pysparkDF_indexed)

# Define same base-scaler for both feature cols
scaler = StandardScaler(inputCol="features",
                        outputCol="scaledFeatures",
                        withStd=True, 
                        withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel_unconditional = scaler.fit(featureSet_unconditional)
scalerModel_conditional = scaler.fit(featureSet_conditional)

scaledFeatureSet_unconditional = scalerModel_unconditional.transform(featureSet_unconditional)
scaledFeatureSet_conditional = scalerModel_conditional.transform(featureSet_conditional)

### Split data into training and test set

In [ ]:
splits_unconditional = scaledFeatureSet_unconditional.randomSplit([0.8, 0.2], 12345)
training_unconditional = splits_unconditional[0]
test_unconditional = splits_unconditional[1]

splits_conditional= scaledFeatureSet_conditional.randomSplit([0.8, 0.2], 12345)
training_conditional = splits_conditional[0]
test_conditional = splits_conditional[1]

In [ ]:
spark.stop()